In [ ]:
!pip install ../input/pytorchcv/pytorchcv-0.0.55-py2.py3-none-any.whl --quiet

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from timeit import default_timer as timer
import skimage.io

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *

import random
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(12)

if True:
    DATA_DIR = '/kaggle/input/prostate-cancer-grade-assessment/'
    SUBMISSION_CSV_FILE = 'submission.csv'

import warnings
warnings.filterwarnings('ignore')

#### net #########################################################################

## load net -----------------------------------
net = []
num_classes = 6

from pytorchcv.model_provider import get_model

#--------------------------------------------------------

from torch.nn.parameter import Parameter
def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps
    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)       
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

class Head(torch.nn.Module):
  def __init__(self, in_f, out_f, dropout):
    super(Head, self).__init__()
    
    self.f = nn.Flatten()
    self.l = nn.Linear(in_f, 512)
#     self.m = Mish()
    self.r = nn.ReLU()
    self.d = nn.Dropout(0.25)
    self.dropout = dropout
#     self.o = nn.Linear(512, out_f)
    self.o = nn.Linear(in_f, out_f)
    self.b1 = nn.BatchNorm1d(in_f)
    self.b2 = nn.BatchNorm1d(512)

  def forward(self, x):
    x = self.f(x)
#     x = self.b1(x)
    if self.dropout:
      x = self.d(x)

#     x = self.l(x)
#     x = self.r(x)
#     x = self.b2(x)
#     if self.dropout:
#       x = self.d(x)

    out = self.o(x)
    return out

class FCN(torch.nn.Module):
  def __init__(self, base, in_f, num_classes, dropout=True):
    super(FCN, self).__init__()
    self.base = base
    self.h1 = Head(in_f, num_classes, dropout)
  
  def forward(self, x):
    x = self.base(x)
    return self.h1(x)

def create_model6():
    model = get_model("efficientnet_b0", pretrained=False)
    features = list(model.children())[-1][-1].in_features
    model = nn.Sequential(*list(model.children())[:-1]) # Remove original output layer
#     model[0].final_pool = nn.Sequential(nn.AdaptiveAvgPool2d(1))
    model[0].final_pool = nn.Sequential(GeM())
    model = FCN(model, features, num_classes - 1, dropout=True)
    return model

def create_model7():
    model = get_model("efficientnet_b1", pretrained=False)
    features = list(model.children())[-1][-1].in_features
    model = nn.Sequential(*list(model.children())[:-1]) # Remove original output layer
#     model[0].final_pool = nn.Sequential(nn.AdaptiveAvgPool2d(1))
    model[0].final_pool = nn.Sequential(GeM())
    model = FCN(model, features, num_classes - 1, dropout=True)
    return model
#------------------------------------------------------

model = create_model6()
model = model.cuda()
state = torch.load('../input/panda-models/model-fld1 (.867).pth') # .89
model.load_state_dict(state['model_state'])
net.append(model)

model = create_model7()
model = model.cuda()
state = torch.load('../input/panda-models/model-fld2 (.866).pth') # .89
model.load_state_dict(state['model_state'])
net.append(model)

model = create_model7()
model = model.cuda()
state = torch.load('../input/panda-models/model-fld1 (.872).pth') # .89
model.load_state_dict(state['model_state'])
net.append(model)

# model = create_model6()
# model = model.cuda()
# state = torch.load('../input/panda-models/model-fld5 (.87).pth') # .89
# model.load_state_dict(state['model_state'])
# net.append(model)

model = create_model6()
model = model.cuda()
state = torch.load('../input/panda-models/model-fld1 (.868).pth') # .89
model.load_state_dict(state['model_state'])
net.append(model)
#------------------------------------------

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import albumentations as A

sz = 256
N = 36

def tile(img):
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    imgs = img[idxs]
    result = []
    for i in range(len(imgs)):
        result.append({'img':imgs[i], 'idx':i})
    return result

import io

class ImageDataset3(Dataset):
    def __init__(self, dataframe, root_dir, transform=None, transform2=None):
        self.df = dataframe
        self.root_dir = root_dir
        self.transform = transform
        self.transform2 = transform2

        self.paths = self.df.image_id.values

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = self.paths[idx]
        img = skimage.io.MultiImage(os.path.join(self.root_dir,img_name+'.tiff'))[1]
#         img = crop_white(img)
#         buffer = io.BytesIO()
#         Image.fromarray(img).save(buffer, format='jpeg', quality=90)
#         img = np.array(Image.open(buffer))
        
        imgs = [img_['img'] for img_ in tile(img)]
        imgs = [(255 - img).astype(np.float32) / 255. for img in imgs]
        
        if self.transform is not None:
          imgs = [self.transform(image=img)['image'] for img in imgs]
        
        imgs = np.array(imgs)
        img_ = []
        idx_ = 0
        for i in range(6):
          img = []
          for j in range(6):
              img.append(imgs[idx_])
              idx_ += 1
          img_.append(np.hstack(np.array(img)))
        imgs = np.vstack(np.array(img_))
        
        if self.transform2 is not None:
          imgs = self.transform2(image=imgs)['image']
        imgs = np.rollaxis(imgs, -1, 0)

        return imgs

class ImageDataset4(Dataset):
    def __init__(self, dataframe, root_dir, transform=None, transform2=None):
        self.df = dataframe
        self.root_dir = root_dir
        self.transform = transform
        self.transform2 = transform2

        self.paths = self.df.image_id.values

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = self.paths[idx]
        img = skimage.io.MultiImage(os.path.join(self.root_dir,img_name+'.tiff'))[1]
        img = np.pad(img, ((sz//2, sz//2), (sz//2, sz//2), (0, 0)), constant_values=255)
#         img = crop_white(img)
#         buffer = io.BytesIO()
#         Image.fromarray(img).save(buffer, format='jpeg', quality=90)
#         img = np.array(Image.open(buffer))
        
        imgs = [img_['img'] for img_ in tile(img)]
        imgs = [(255 - img).astype(np.float32) / 255. for img in imgs]
        
        if self.transform is not None:
          imgs = [self.transform(image=img)['image'] for img in imgs]
        
        imgs = np.array(imgs)
        img_ = []
        idx_ = 0
        for i in range(6):
          img = []
          for j in range(6):
              img.append(imgs[idx_])
              idx_ += 1
          img_.append(np.hstack(np.array(img)))
        imgs = np.vstack(np.array(img_))
        
        if self.transform2 is not None:
          imgs = self.transform2(image=imgs)['image']
        imgs = np.rollaxis(imgs, -1, 0)

        return imgs

#---------------------------------------------

tta1 = A.Compose([
    A.VerticalFlip(p=1)
])

tta2 = A.Compose([
    A.HorizontalFlip(p=1),
])

tta3 = A.Compose([
    A.Transpose(p=1),
])

train = pd.read_csv(f'{DATA_DIR}train.csv')[:11]
# submission = train
submission = pd.read_csv(f'{DATA_DIR}sample_submission.csv')

def run_make_submission_csv():
    target=[]
    single = False
    m1,m2,m3,m4,m5 = [],[],[],[],[]
    batch_size= 2

    if os.path.exists('../input/prostate-cancer-grade-assessment/test_images'):
#     if True:
        image_path = f'{DATA_DIR}test_images/'
#         image_path = f'{DATA_DIR}train_images/'

        test_dataset2 = ImageDataset3(submission, image_path, None)
        test_loader2 = DataLoader(dataset=test_dataset2, batch_size=batch_size, shuffle=False, num_workers=4)
        # tta
        test_dataset3 = ImageDataset3(submission, image_path, transform2=tta1)
        test_loader3 = DataLoader(dataset=test_dataset3, batch_size=batch_size, shuffle=False, num_workers=4)
        
        test_dataset4 = ImageDataset3(submission, image_path, transform2=tta2)
        test_loader4 = DataLoader(dataset=test_dataset4, batch_size=batch_size, shuffle=False, num_workers=4)
        
#         test_dataset5 = ImageDataset4(submission, image_path, None)
#         test_loader5 = DataLoader(dataset=test_dataset5, batch_size=batch_size, shuffle=False, num_workers=4)
        
        test_dataset6 = ImageDataset3(submission, image_path, transform2=tta3)
        test_loader6 = DataLoader(dataset=test_dataset6, batch_size=batch_size, shuffle=False, num_workers=4)
        
        t2 = tqdm(test_loader2)
        dataloader_iterator1 = iter(test_loader3)
        dataloader_iterator2 = iter(test_loader4)
#         dataloader_iterator3 = iter(test_loader5)
        dataloader_iterator4 = iter(test_loader6)
        with torch.no_grad():
            for b, image_batch in enumerate(t2):
                image_batches = []
                image_batch = image_batch.cuda().float()
                try:
                    image_batch2 = next(dataloader_iterator1).cuda().float()
                    image_batch3 = next(dataloader_iterator2).cuda().float()
#                     image_batch4 = next(dataloader_iterator3).cuda().float()
                    image_batch5 = next(dataloader_iterator4).cuda().float()
                except:
                    dataloader_iterator1 = iter(test_loader3)
                    dataloader_iterator2 = iter(test_loader4)
#                     dataloader_iterator3 = iter(test_loader5)
                    dataloader_iterator4 = iter(test_loader6)
                    image_batch2 = next(dataloader_iterator1).cuda().float()
                    image_batch3 = next(dataloader_iterator2).cuda().float()
#                     image_batch4 = next(dataloader_iterator3).cuda().float()
                    image_batch5 = next(dataloader_iterator4).cuda().float()
                image_batches = [image_batch,image_batch2,image_batch3,image_batch5]
                
                if single:
                    model = net[0].eval()
                    o1 = model(image_batch)
                    preds = []
                    for l in o1:
                        preds.append(len(np.where(np.round(F.sigmoid(l).cpu())==1)[0]))
                    target.append(preds)
                else:
                    # tta
                    all_preds = []
                    for i,model in enumerate(net):
                        for image_batch in image_batches:
                            model = model.eval()
                            o1 = model(image_batch)
                            preds = []
                            for l in o1:
                                preds.append(F.sigmoid(l).cpu().numpy())
                            all_preds.append(preds)
                        
                    all_preds = np.array(all_preds)
                    all_preds = np.rollaxis(all_preds, 0, 2)
                    pred= np.mean(all_preds, axis=1)
                    
                    for p in pred:
                        target.append(len(np.where(np.round(p)==1)[0]))
                    
    #---------
    else:
        if single:
            target = [[1],[1],[1]]
        else:
            target = [1,1,1]
    if single:
        target = np.concatenate(target)
    
    submission['isup_grade'] = target
    submission['isup_grade'] = submission['isup_grade'].astype(int)
    submission.to_csv(SUBMISSION_CSV_FILE, index=False)
    print(submission.head(25))

if __name__ == '__main__':
    run_make_submission_csv()

    print('\nsucess!')